Which files are the most popular by downloads * size?
====

Query most popular files.

```
SELECT * FROM (
  SELECT url, count(*) as downloads FROM `the-psf.pypi.file_downloads`  
  WHERE DATE(timestamp) = "2020-04-16" GROUP by url
) ORDER BY downloads DESC
LIMIT 1000
```

Download query as json. Attach Content-Length to each file.

```
prefix = 'https://files.pythonhosted.org'
for item in data: 
    url = item['url'] 
    content = session.head(prefix + url) 
    length = content.headers['Content-Length'] 
    item['length'] = int(length)
    item['downloads'] = int(item['downloads'])
```

In [27]:
import json
data = json.load(open('pypipopular.json'))
for item in data:
    item['aggregate'] = item['length'] * item['downloads']
data.sort(key=lambda x: -x['downloads'])
for i, item in enumerate(data): item['rank'] = i+1 # ranked by number of downloads
data.sort(key=lambda x: -x['aggregate']) # sorted by downloads * file size

In [28]:
rows = []
for item in data[:128]:
    rows.append((item['url'].split('/')[-1], "{:.0f}".format(item['aggregate']/2**30), item['rank']))

In [29]:
table = ("<table>" + "<tr><th>file</th><th>GiB/day</th><th>popularity</th></tr>" + "\n".join("<tr><td>{}</td><td>{}</td><td>{}</td></tr>".format(*row) for row in rows) + "</table>")
import IPython.display
IPython.display.HTML(table)

file,GiB/day,popularity
pyspark-2.4.5.tar.gz,15345,369
tensorflow-2.1.0-cp36-cp36m-manylinux2010_x86_64.whl,15083,643
tensorflow-2.1.0-cp37-cp37m-manylinux2010_x86_64.whl,9552,905
tensorflow-2.0.0-cp27-cp27mu-manylinux2010_x86_64.whl,8004,298
botocore-1.15.39-py2.py3-none-any.whl,7769,12
numpy-1.18.2-cp36-cp36m-manylinux1_x86_64.whl,7001,60
scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl,4243,177
xgboost-1.0.2-py3-none-manylinux1_x86_64.whl,3488,705
scipy-1.1.0-cp27-cp27mu-manylinux1_x86_64.whl,3200,269
numpy-1.18.2-cp37-cp37m-manylinux1_x86_64.whl,3096,185
